In [49]:
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
import datetime
import time
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

import re

data_de_hoje = datetime.date.today()

# Coleta os produtos da loja
produtos = []

# Variáveis
service = ChromeService(
    ChromeDriverManager(driver_version="137.0.7151.104").install()
)
driver = webdriver.Chrome(service=service)

driver.get("https://obahortifruti.com.br/")

time.sleep(1)

precos_oba = []

wait = WebDriverWait(driver, 10)

try:
    botao_localizacao = wait.until(EC.element_to_be_clickable((By.XPATH, "//section[contains(@class, 'styles_geolocationTrigger')]/button")))
    botao_localizacao.click()
    print("Botão clicado com sucesso!")
except Exception as e:
    print("Erro ao tentar clicar no botão:", e)

    # Preencher o CEP
try:
    campo_cep = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@placeholder='99999-999']")))
    campo_cep.clear()
    campo_cep.send_keys("70675710")
    print("CEP preenchido com sucesso!")
except Exception as e:
    print("Erro ao preencher o CEP:", e)

# Clicar no botão "Buscar"
try:
    botao_buscar = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@data-submit-btn='true']")))
    botao_buscar.click()
    print("Botão Buscar clicado com sucesso!")
except Exception as e:
    print("Erro ao clicar no botão Buscar:", e)

try:
    botao_retirar = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Retirar em loja')]")))
    botao_retirar.click()
    print("Botão 'Retirar em loja' clicado com sucesso!")
except Exception as e:
    print("Erro ao clicar no botão 'Retirar em loja':", e)


print("Fim etapa de abertura")


Botão clicado com sucesso!
CEP preenchido com sucesso!
Botão Buscar clicado com sucesso!
Botão 'Retirar em loja' clicado com sucesso!
Fim etapa de abertura


In [50]:
# Passo 5: Coletar todas as lojas listadas
wait.until(EC.presence_of_element_located((By.XPATH, "//ul[@data-pickup-selector='true']")))
ul_lojas = driver.find_element(By.XPATH, "//ul[@data-pickup-selector='true']")
lojas = ul_lojas.find_elements(By.TAG_NAME, "li")

dados_lojas = []
dados_produtos = []

for index in range(len(lojas)):
    try:
        # Recarrega a lista e faz scroll até o elemento visível
        lojas = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//ul[@data-pickup-selector='true']/li")))
        ul_lojas = driver.find_element(By.XPATH, "//ul[@data-pickup-selector='true']")
        driver.execute_script("arguments[0].scrollTop = arguments[0].children[arguments[1]].offsetTop", ul_lojas, index)
        time.sleep(0.5)

        # Pega o texto da loja e divide
        texto_loja = lojas[index].text.strip()
        partes = texto_loja.split('\n')
        nome_loja = partes[0] if len(partes) > 0 else ''
        endereco = partes[1] if len(partes) > 1 else ''
        bairro_cidade = partes[2] if len(partes) > 2 else ''
        print(f"\nSelecionando loja: {nome_loja}")

        # Clicar na loja
        time.sleep(0.5)
        wait.until(EC.element_to_be_clickable(lojas[index])).click()

        # Clicar no botão "Continuar"
        time.sleep(0.5)
        # try:
        #     botao_continuar = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[@data-continue-btn='true']")))
        #     botao_continuar.click()
        #     print("Botão 'Continuar' clicado com sucesso!")
        # except Exception as e:
        #     print("Erro ao clicar em 'Continuar':", e)
        #     continue

        # Clicar no botão "Continuar" de forma segura (sem guardar referência)
        try:
            xpath_continuar = "//button[@data-continue-btn='true']"
            
            for tentativa in range(3):
                try:
                    print(f"Tentativa {tentativa + 1}: clicando no botão 'Continuar'...")
                    wait.until(EC.presence_of_element_located((By.XPATH, xpath_continuar)))
                    driver.execute_script("""
                        document.evaluate(arguments[0], document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null)
                                .singleNodeValue.click();
                    """, xpath_continuar)
                    print("Botão 'Continuar' clicado com sucesso!")
                    break
                except Exception as e:
                    print(f"Tentativa {tentativa + 1}: erro ao clicar em 'Continuar': {e}")
                    time.sleep(1)
        except Exception as e:
            print("Erro final ao tentar clicar no botão 'Continuar':", e)


        # Digitar o nome do produto
        time.sleep(0.5)
        try:
            campo_busca = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Digite seu produto']")))
            campo_busca.clear()
            campo_busca.send_keys("cerveja")
            print("Termo 'cerveja' digitado com sucesso!")
        except Exception as e:
            print("Erro ao digitar o produto:", e)
        

        # Clicar na lupa via JavaScript
        time.sleep(0.5)
        try:
            botao_lupa = wait.until(EC.presence_of_element_located((By.XPATH, "//span[@data-submit-search='true']")))
            driver.execute_script("arguments[0].click();", botao_lupa)
            print("Busca iniciada!")
        except Exception as e:
            print("Erro ao clicar na lupa:", e)



        time.sleep(0.5)
        try:
            campo_busca_modal = wait.until(EC.visibility_of_element_located(
                (By.XPATH, "//input[@data-testid='fs-input']")))
            campo_busca_modal.clear()
            campo_busca_modal.send_keys("cerveja")
            print("Termo 'cerveja' digitado com sucesso no campo correto!")
        except Exception as e:
            print("Erro ao preencher o campo de busca:", e)

        ### ESTRANHO MAS DEU CERTO
        time.sleep(0.5)
        try:
            botao_lupa_span = wait.until(EC.presence_of_element_located(
                (By.XPATH, "//span[@data-submit-search='true']")))
            driver.execute_script("arguments[0].click();", botao_lupa_span)
            print("Clique na lupa executado via JavaScript com sucesso!")
        except Exception as e:
            print("Erro ao clicar na lupa via JavaScript:", e)

        # # Coletar os produtos
        # try:
        #     produtos = wait.until(EC.presence_of_all_elements_located((By.XPATH, "//article")))
        #     for produto in produtos:
        #         try:
        #             nome_produto = produto.find_element(By.XPATH, ".//a[@data-testid='product-link']").get_attribute("title")
        #         except:
        #             nome_produto = "Nome não encontrado"

        #         try:
        #             preco = produto.find_element(By.XPATH, ".//div[@data-price-per-unit='true']").text
        #         except:
        #             preco = "Preço não encontrado"

        #         dados_produtos.append({
        #             "produto": nome_produto,
        #             "preco": preco,
        #             "nome_loja": nome_loja,
        #             "endereco": endereco,
        #             "bairro_cidade": bairro_cidade
        #         })
        #     print(f"{len(produtos)} produtos coletados na loja '{nome_loja}'")
        # except Exception as e:
        #     print("Erro ao coletar os produtos:", e)


        # Clica no botão de seleção de loja (botão com ícone de localização e nome da loja atual)
        # Clica no botão de seleção de loja (usando clique via JavaScript para evitar erros silenciosos)
        # Clica no botão de seleção de loja (com retry em caso de stale element)
        time.sleep(0.5)
        # Clica no botão de seleção de loja com abordagem 100% resistente a 'stale element'
        try:
            xpath_botao = "//button[.//span[contains(text(),'Oba')]]"

            for tentativa in range(3):
                try:
                    print(f"Tentativa {tentativa + 1}: tentando localizar e clicar no botão...")
                    # Aqui buscamos e clicamos imediatamente no botão, sem guardar ele entre um e outro
                    wait.until(EC.presence_of_element_located((By.XPATH, xpath_botao)))
                    driver.execute_script("""
                        document.evaluate(arguments[0], document, null, XPathResult.FIRST_ORDERED_NODE_TYPE, null)
                                .singleNodeValue.click();
                    """, xpath_botao)
                    print("Botão para abrir a lista de lojas clicado com sucesso!")
                    break
                except Exception as e:
                    print(f"Tentativa {tentativa + 1}: erro ao clicar no botão: {e}")
                    time.sleep(1)

        except Exception as e:
            print("Erro final ao tentar abrir lista de lojas:", e)





        ### Clicar no botao "Retire na loja"
        time.sleep(0.5)
        try:
            botao_retirar = wait.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(text(), 'Retirar em loja')]")))
            botao_retirar.click()
            print("Botão 'Retirar em loja' clicado com sucesso!")
        except Exception as e:
            print("Erro ao clicar no botão 'Retirar em loja':", e)


    except Exception as e:
        print(f"Erro geral na loja {index+1}: {e}")
        continue

# Salvar os dados em CSV
df_produtos = pd.DataFrame(dados_produtos)
df_produtos.to_csv("produtos_por_loja.csv", index=False, encoding='utf-8-sig')
print("\nColeta finalizada e arquivo 'produtos_por_loja.csv' salvo com sucesso!")



Selecionando loja: Oba 302 Sudoeste
Tentativa 1: clicando no botão 'Continuar'...
Botão 'Continuar' clicado com sucesso!
Termo 'cerveja' digitado com sucesso!
Busca iniciada!
Termo 'cerveja' digitado com sucesso no campo correto!
Clique na lupa executado via JavaScript com sucesso!
Tentativa 1: tentando localizar e clicar no botão...
Botão para abrir a lista de lojas clicado com sucesso!
Botão 'Retirar em loja' clicado com sucesso!

Selecionando loja: Oba 105 Sul
Tentativa 1: clicando no botão 'Continuar'...
Botão 'Continuar' clicado com sucesso!
Termo 'cerveja' digitado com sucesso!
Busca iniciada!
Erro ao preencher o campo de busca: Message: 
Stacktrace:
0   chromedriver                        0x0000000104dedd14 cxxbridge1$str$ptr + 2735276
1   chromedriver                        0x0000000104de5f88 cxxbridge1$str$ptr + 2703136
2   chromedriver                        0x00000001049366f0 cxxbridge1$string$len + 90424
3   chromedriver                        0x000000010497d9e0 cxxbridge1